### how to solve inter hyphen issue https://stackoverflow.com/questions/52293874/why-does-spacy-not-preserve-intra-word-hyphens-during-tokenization-like-stanford

In [157]:
import re
import os
import json
import spacy
import nltk
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from benepar.spacy_plugin import BeneparComponent
from spacy.tokens import Doc
nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent('benepar_en2'))

In [97]:
# def custom_tokenizer(nlp):
# #     infix_re = re.compile(r'''[~-]''')
#     infix_re = compile_infix_regex(nlp.Defaults.infixes)
#     prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
#     suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)

#     return Tokenizer(nlp.vocab, 
#                      prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 token_match=None)


def custom_tokenizer(text):
    tokens = nltk.word_tokenize(text)
    # with this:
    return Doc(nlp.vocab, tokens)

nlp.tokenizer = custom_tokenizer

In [193]:
def get_doc(doc_path):
    with open(doc_path) as f:
        txt = f.read().replace('\n', ' ')
    doc = nlp(txt)
    sentences = []
    sent_token_offset = 0
    for i, sent in enumerate(doc.sents):
        if i == 0: continue
        words = []
        dependencies = []
        count = 0
        for word in sent:
            charoffsetbegin = word.idx
            charoffsetend = word.idx + len(word)
            word_pos = word.pos_
            word_string = str(word)
            if str(word) == '\"':
                if count % 2 == 0:
                    word_string = '``'
                else:
                    word_string = '\'\''
            word_combo = [word_string, 
                          {"CharacterOffsetBegin": charoffsetbegin, 
                                      "CharacterOffsetEnd": charoffsetend, 
                                      "Linkers": [], 
                                      "PartOfSpeech": word.pos_}]
            words.append(word_combo)

            dep_r = word.dep_
            parent = str(word.head) + '-' + str(word.head.i - sent_token_offset)
            child = str(word) + '-' + str(word.i - sent_token_offset)
            dep = [dep_r, parent, child]
            dependencies.append(dep)

        parsetree = sent._.parse_string
        parsetree = re.sub(r'\(_SP ((.|\n)*)\)', '', parsetree)
        sentences.append({"parsetree": parsetree, "words": words, "dependencies": dependencies})
        sent_token_offset += len(sent)
    return sentences

In [194]:
folder_path = '/home/pengfei/data/pdtb3-dataset/all/raw/'
files = os.listdir(folder_path)
all_data = {}
files = ['wsj_1094']
for f in files:
    doc_path = os.path.join(folder_path, f)
    doc = get_doc(doc_path)
    all_data[f] = {'sentences': doc}

In [170]:
# with open('pdtb-parse.json', 'w') as f:
#     json.dump(all_data, f)

In [171]:
json_path = '/home/pengfei/data/pdtb3-dataset/all/conll/train/pdtb-parses.json'
with open(json_path,  'r') as f:
    parse_dict = json.load(f)

In [172]:
def replace_quotation(input_list):
    count = 0
    for i,o in enumerate(input_list):
        if o == '"':
            if count % 2 == 0:
                input_list[i] = '``'
            else:
                input_list[i] = '\'\''
        count += 1
    return input_list

In [173]:
for f in files:
    if f not in parse_dict:
        continue 
    predict = all_data[f]
    gold = parse_dict[f]
    predict_s_index = 0
    gold_s_index = 0
    
    while predict_s_index <= len(predict['sentences']) and gold_s_index <= len(gold['sentences']):
        gold_token = [o[0] for o in gold['sentences'][gold_s_index]['words']]
        predict_token = [o[0] for o in predict['sentences'][predict_s_index]['words']]
        if gold_token != predict_token:
            print(gold_s_index, predict_s_index)
            predict_token.extend([o[0] for o in predict['sentences'][predict_s_index + 1]['words']])
            if gold_token == predict_token:
                gold_s_index += 1
                predict_s_index += 2
            else:
                gold_s_index += 1
                predict_s_index += 1
                print('predict_token')
                print(' '.join(predict_token))
                print('\n')
                print('gold_token')
                print(' '.join(gold_token))
                print('\n')
                print('-----')
        else:
            gold_s_index += 1
            predict_s_index += 1
    break
#     for i, (predict_s, gold_s) in enumerate(zip(predict['sentences'], gold['sentences'])):
#         gold_token = [o[0] for o in gold_s['words']]
#         predict_token = [o[0] for o in predict_s['words']]
#         if '\n' in predict_token[-1]:
#             predict_token.pop()
#         predict_token = replace_quotation(predict_token)
#         if gold_token != predict_token:
#             print(i)
#             print('gold_token: ')
#             print(' '.join(gold_token))
# #             print(gold_s['parsetree'])
#             print('\n')
#             print('predict_token')
#             print(' '.join(predict_token))
# #             print(predict_s['parsetree'])
#             print('\n\n')
#             print('-----')
#     break

1 1
predict_token
What did he think of the Supreme Court 's decision legalizing abortion ?    " I happen to think it was right , " Mr. Bush said flatly .   


gold_token
What did he think of the Supreme Court 's decision legalizing abortion ?


-----
2 2
predict_token
" I happen to think it was right , " Mr. Bush said flatly .    A few months later , Mr. Bush became Ronald Reagan 's running mate .


gold_token
`` I happen to think it was right , '' Mr. Bush said flatly .


-----
4 4
predict_token
Suddenly , George Bush the pro - choice advocate became George Bush the anti - abortionist . And the vacillation did n't end there .   


gold_token
Suddenly , George Bush the pro-choice advocate became George Bush the anti-abortionist .


-----
5 5
predict_token
And the vacillation did n't end there .    Just a month ago , Mr. Bush sternly threatened to veto a pending welfare bill if it provided any abortion funds , except to save a woman 's life .


gold_token
And the vacillation did n't end

IndexError: list index out of range